In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, col, lower, regexp_replace, split, to_date

spark= SparkSession.builder.config('spark.ui.port','4040').getOrCreate()

In [2]:
journal = spark.read.csv("file:///N/project/mag/mag-2021-01-05/mag/Journals.txt",\
                         header = False,\
                         sep=r'\t')
#journal.printSchema()

In [3]:
def clean_text(c):
  c = regexp_replace(c, '"' , '')
  c = regexp_replace(c, '\\\\' , '')
  #c = regexp_replace(c, "(https?\://)\S+", "")
  #c = regexp_replace(c, "[^a-zA-Z0-9\\s]", "")
  #c = split(c, "\\s+") tokenization...
  return c

In [4]:
journal = journal.select(
                         journal._c0.cast("string").alias("journalId"),
                         journal._c1.cast("int").alias("rank"),
                         journal._c2.cast("string").alias("normalizedName"),
                         clean_text(col("_c3")).alias("displayName"),
                         journal._c4.cast("string").alias("issn"),
                         journal._c5.cast("string").alias("publisher"),
                         journal._c6.cast("string").alias("webpage"),
                         journal._c7.cast("int").alias("paperCount"),
                         journal._c8.cast("int").alias("paperFamilyCount"),
                         journal._c9.cast("int").alias("citationCount"),
                         to_date(journal._c10, "yyyy-MM-dd").alias("createdDate")
)

#journal.show(5)

In [5]:
%%time

journal.coalesce(64).write.option("header","true") \
                               .option("sep","\t") \
                               .option("quoteAll", False) \
                               .option("emptyValue", None) \
                               .option("nullValue", None)\
                               .mode("overwrite") \
                               .csv('/N/project/mag/mag_jg_2021_update/nodes/journal')               

CPU times: user 6.54 ms, sys: 1.37 ms, total: 7.9 ms
Wall time: 2.32 s
